## Notebook to bulk download Planet imagery through the API
Rainey Aberle, Fall 2021

Modified from [Planet Developers API Tutorial](https://developers.planet.com/docs/apis/data/) and [Planet Labs: `ordering_and_delivery.ipynb`](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/orders/ordering_and_delivery.ipynb )


__To-Do:__
- View image thumbnails before downloading. Will need to download only the metadata first, then plot the coordinates over a regional map. 
- Implement percent AOI coverage filter for image search

### Import necessary packages

In [83]:
import os
import json
import requests
import time
import geopandas as gpd
from pathlib import Path
import rasterio as rio
import numpy as np
from rasterio.plot import show
from requests.auth import HTTPBasicAuth

### Install Planet API Client

This will allow you to interact with the Planet API through this notebook. Refer to the __[Planet API documentation](https://developers.planet.com/docs/apis/data/)__ for more info. 

In [13]:
# !conda install -c conda-forge rasterio -y

### Define filters for image search
#### _Modify these sections_

In [84]:
#### OPTION 2: Import an existing shapefile

# Name of your file
# If your shapefile is not currently in this directory, you need to include the full file path in 'file_name' below
file_name = '/Volumes/SGlacier/BoxTurner/BoxTurner.shp'

# Read in the shapefile
AOI = gpd.read_file(file_name)

# File extension index (we don't want the .shp extension in the next line)
i = file_name.index('.shp')

# Convert to geojson
AOI.to_file(file_name[0:i]+'.geojson', driver='GeoJSON')

# Adjust AOI polygon to a rectangular shape 
# Planet only excepts a bounding BOX as a spatial filter, 
# so we need to convert our AOI to a box (if it is not already). 
AOI_box = {"type": "Polygon",
           "coordinates": [[
               [AOI.bounds.minx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.miny[0]]
           ]]
          }
AOI_box

/opt/miniconda3/envs/autorift/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


{'type': 'Polygon',
 'coordinates': [[[-140.1698068780743, 59.904177627649],
   [-139.42646300960544, 59.904177627649],
   [-139.42646300960544, 60.27836331801325],
   [-140.1698068780743, 60.27836331801325],
   [-140.1698068780743, 59.904177627649]]]}

In [85]:
# ----------AOI clipping----------
# Would you like to clip images to the AOI (True/False)?
clip_AOI = True

# ----------Date Range----------
# Format: 'YYYY-MM-DD'
start_date = "2021-01-01"
end_date = "2021-01-31"

# ----------Cloud Filter----------
# Format: decimal (e.g., 50% max cloud cover = 0.5)
max_cloud_cover = 0.8

# ----------Item Type----------
# See here for possible image ("item") types:
# https://developers.planet.com/docs/apis/data/items-assets/
item_type = "PSScene4Band"

# ----------Planet API Key----------
# Find your API key on your Planet Account > My Settings > API Key
API_key = 'c2e92a042f6744eba732c282d09539f8'

# ----------Output folder----------
# AKA, where you want your images to be downloaded in your directory
out_folder = '/Volumes/SGlacier/Turner20_21/01_21/'

### Authentication via basic HTTP

In [86]:
# set API key as environment variable
os.environ['PL_API_KEY'] = API_key

# Setup the API Key stored as the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Orders URL
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# Authorize
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

<Response [200]>

### Compile filters and use Quick Search to grab image IDs

In [87]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": AOI_box
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": start_date + "T00:00:00.000Z",
    "lte": end_date + "T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": max_cloud_cover
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

# define the clip tool
clip = {
    "clip": {
        "aoi": AOI_box
    }
}

# API request object
QS_request = {
  #"interval":"day",
  "item_types":[item_type], 
  "filter":combined_filter,
}

# fire off the POST request
QS_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=QS_request)

# Print resulting image IDs
im_ids = [feature['id'] for feature in QS_result.json()['features']]
print(im_ids)

['20210128_203808_37_2401', '20210125_204310_80_2426', '20210130_201529_1010', '20210130_201528_1010', '20210130_201527_1010', '20210130_201526_1010', '20210130_201525_1010', '20210130_201524_1010', '20210130_201523_1010', '20210130_204356_43_2402', '20210130_204354_16_2402', '20210130_204351_89_2402', '20210130_204349_61_2402', '20210130_203417_75_2416', '20210130_203415_40_2416', '20210130_203413_05_2416', '20210130_203410_70_2416', '20210130_204028_31_227a', '20210130_204035_42_227a', '20210130_204033_05_227a', '20210130_204030_68_227a', '20210127_204843_80_1057', '20210127_204845_31_1057', '20210128_204902_68_1066', '20210127_200901_1003', '20210127_200900_1003', '20210127_204851_35_1057', '20210127_204849_84_1057', '20210127_204848_33_1057', '20210127_204846_82_1057', '20210127_201812_1025', '20210127_201811_1025', '20210126_204908_31_105d', '20210126_204911_33_105d', '20210126_204909_82_105d', '20210126_205747_71_1060', '20210126_204222_92_2414', '20210126_204220_64_2414', '20210

### Place Order

In [88]:
# set content type to json
headers = {'content-type': 'application/json'}

# create a request object
# (clip images if clip==True)
if clip_AOI:
    request = {  
       "name":"simple order",
       "products":[
          {
              "item_ids": im_ids,
              "item_type": item_type,
              "product_bundle":"analytic_sr" # CHANGE PRODUCT BUNDLE
          }
       ],
        "tools": [clip]
    }
else:
    request = {  
       "name":"simple order",
       "products":[
          {
              "item_ids": im_ids,
              "item_type": item_type,
              "product_bundle":"analytic_sr" # CHANGE PRODUCT BUNDLE
          }
       ],
    }

    # Harmonize
{
  "name": "simple order",
  "products":[
          {
              "item_ids": im_ids,
              "item_type": item_type,
              "product_bundle":"analytic_sr" # CHANGE PRODUCT BUNDLE
          }
       ],
  "tools": [
    {
      "harmonize": {
        "target_sensor": "Sentinel-2"
      }
    }
  ]
}

# define function to place order
def place_order(search_request, auth):
    response = requests.post(orders_url, data=json.dumps(search_request), auth=auth, headers=headers)
# #     if True == True:
    if not 'No access to assets' in str(response.json()): # if no access error
#         print(response.json())
        order_id = response.json()['id']
        print(order_id)
        order_url = orders_url + '/' + order_id
        return order_url
    else:
        responsedict = response.json()['field']
#         print(responsedict) # if errors, check the response
        badids = []
        for message in responsedict['Details']:
            message = message['message'] # grab the message
            badid = message[len('No access to assets: ')+len(item_type):].split('/')[1] # split the access message
            #2nd item (index 1) is the ID corresponding to the item causing the error
            badids.append(badid)
        print('Remove these im_ids and re-run:')
        return badids

In [89]:
# place order
order_url = place_order(request, auth)
print(order_url)

d5a72ba3-a641-42ae-892a-52f15979c781
https://api.planet.com/compute/ops/orders/v2/d5a72ba3-a641-42ae-892a-52f15979c781


In [63]:
###################### REMOVE BAD IDs (no access) ######################
for im_id in order_url:
#     print(im_id)
    im_ids.remove(im_id)
########################################################################

### Poll for Order Success
- This section outputs the status of the order every ~10 sec. This will take a few minutes... 
- Wait until it outputs `success` to proceed to the next section. It will stop after 30 loops, so try proceeding to the next section if it finishes running and does not output `success`.
- If you are ordering a LOT of images, consider narrowing your date range to download less images at a time. 

In [90]:
def poll_for_success(order_url, auth, sleep_time):
    count = 0
#     while(count < num_loops):
    while True == True: # run until end state is reached
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        time.sleep(sleep_time)
        if state in end_states:
            break
    print(state)
        
poll_for_success(order_url, auth, 30)

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
success
success


### View Results

In [91]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']

[r['name'] for r in results]

['d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_3B_AnalyticMS_SR_clip.tif',
 'd5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_metadata.json',
 'd5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_3B_AnalyticMS_metadata_clip.xml',
 'd5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_3B_AnalyticMS_DN_udm_clip.tif',
 'd5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201528_1010_metadata.json',
 'd5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201528_1010_3B_AnalyticMS_SR_clip.tif',
 'd5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201528_1010_3B_AnalyticMS_DN_udm_clip.tif',
 'd5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201528_1010_3B_AnalyticMS_metadata_clip.xml',
 'd5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201529_1010_metadata.json',
 'd5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201529_1010_3B_AnalyticMS_metadata_clip.xml',


### Download each asset individually

In [92]:
# define function to download results
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = Path(os.path.join(out_folder,name)) #pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
# download images!
download_results(results)

193 items to download
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_3B_AnalyticMS_SR_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_3B_AnalyticMS_SR_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_metadata.json to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_metadata.json
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_3B_AnalyticMS_metadata_clip.xml to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_3B_AnalyticMS_metadata_clip.xml
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_3B_AnalyticMS_DN_udm_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201527_1010_3B_AnalyticMS_DN_udm_clip.tif
do

downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210126_204912_84_105d_3B_AnalyticMS_DN_udm_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210126_204912_84_105d_3B_AnalyticMS_DN_udm_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210126_204912_84_105d_metadata.json to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210126_204912_84_105d_metadata.json
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210127_201812_1025_metadata.json to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210127_201812_1025_metadata.json
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210127_201812_1025_3B_AnalyticMS_SR_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210127_201812_1025_3B_AnalyticMS_SR_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781

downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_203413_05_2416_metadata.json to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_203413_05_2416_metadata.json
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_203413_05_2416_3B_AnalyticMS_SR_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_203413_05_2416_3B_AnalyticMS_SR_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_203413_05_2416_3B_AnalyticMS_metadata_clip.xml to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_203413_05_2416_3B_AnalyticMS_metadata_clip.xml
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_203413_05_2416_3B_AnalyticMS_DN_udm_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_203413_05_2416_3B_AnalyticMS_DN_udm_clip.tif


downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210125_204303_99_2426_3B_AnalyticMS_metadata_clip.xml to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210125_204303_99_2426_3B_AnalyticMS_metadata_clip.xml
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210125_204303_99_2426_3B_AnalyticMS_SR_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210125_204303_99_2426_3B_AnalyticMS_SR_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210125_204303_99_2426_3B_AnalyticMS_DN_udm_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210125_204303_99_2426_3B_AnalyticMS_DN_udm_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_204354_16_2402_3B_AnalyticMS_DN_udm_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_204354_16_240

downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201523_1010_3B_AnalyticMS_DN_udm_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201523_1010_3B_AnalyticMS_DN_udm_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201523_1010_3B_AnalyticMS_metadata_clip.xml to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210130_201523_1010_3B_AnalyticMS_metadata_clip.xml
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210126_205744_70_1060_3B_AnalyticMS_SR_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210126_205744_70_1060_3B_AnalyticMS_SR_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210126_205744_70_1060_3B_AnalyticMS_DN_udm_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210126_205744_70_1060_3B_Analyti

downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210125_204308_53_2426_3B_AnalyticMS_DN_udm_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210125_204308_53_2426_3B_AnalyticMS_DN_udm_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210128_203808_37_2401_metadata.json to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210128_203808_37_2401_metadata.json
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210128_203808_37_2401_3B_AnalyticMS_SR_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210128_203808_37_2401_3B_AnalyticMS_SR_clip.tif
downloading d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210128_203808_37_2401_3B_AnalyticMS_DN_udm_clip.tif to /Volumes/SGlacier/Turner20_21/01_21/d5a72ba3-a641-42ae-892a-52f15979c781/PSScene4Band/20210128_203808_37_2401_3B_AnalyticMS_DN_udm_clip.tif
down

### Visualize downloaded images

In [ ]:
# # define helpful functions for visualizing downloaded imagery
# def show_rgb(img_file):
#     with rio.open(img_file) as src:
#         b,g,r,n = src.read()

#     rgb = np.stack((r,g,b), axis=0)
#     show(rgb/rgb.max())
    
# def show_gray(img_file):
#     with rio.open(img_file) as src:
#         g = src.read(1)
#     show(g/g.max())
    
# # Replace this path with your image file path
# img_file = out_folder+'7d783b6c-3944-441b-b30d-a3e6834e0174/PSScene4Band/20211004_202212_47_2223_3B_AnalyticMS_clip.tif'
# show_rgb(img_file)

## You did it!

<div>
<img src="sandy-cheeks.jpeg" width="400"/>
</div>